<a href="https://colab.research.google.com/github/MatheusPiassiC/redes_neurais/blob/main/ann_exerc_01_keras_intro(Matheus_PIassi).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Curso: Redes Neurais e Deep Learning

Prof. Denilson Alves Pereira
https://sites.google.com/ufla.br/denilsonpereira/
Departamento de Ciência da Computação -
Instituto de Ciências Exatas e Tecnológicas -
Universidade Federal de Lavras

# Atividade Prática 01

**Instruções:**
1. Siga os passos indicados em cada célula abaixo para completar a atividade.
2. Você deve inserir código somente entre as linhas marcadas com **INICIE O CÓDIGO AQUI** e **TERMINE O CÓDIGO AQUI**. Há uma indicação de quantas linhas de código são necessárias.
3. Em alguns pontos, confira o resultado esperado conforme marcado com **SAÍDA ESPERADA**.

**Tempo estimado para execução**: 1 hora

Versão: Junho, 2021

## O Problema a ser Resolvido

O objetivo da atividade é elaborar uma rede neural para predizer se um paciente tem ou não diabetes, com base nas medidas diagnósticas contidas no *dataset* disponível em https://www.kaggle.com/uciml/pima-indians-diabetes-database.

Os dados são de pacientes do sexo feminino, com pelo menos 20 anos de idade. Os atributos das condições médicas incluem o número de gestações que a paciente teve, seu IMC, nível de insulina, idade e outros. A classe a ser predita é o atributo "Outcome", cujos valores são 0 (não tem diabetes) ou 1 (tem diabetes). Portanto, é um problema de classificação binária.

Você vai praticar as seguintes habilidades:
- Efetuar o pré-processamento dos dados, separando-os em conjuntos de treino e teste.
- Configurar uma rede neural simples para um problema de classificação binária.

## Pacotes

In [ ]:
import numpy as np # package for scientific computing
import tensorflow as tf  #  package for numerical computation using data flow graphs
from tensorflow import keras  # package for deep learning
import pandas as pd # package for working with structured data

## Pré-Processamentos dos Dados de Treino e de Teste

In [ ]:
# Read dataset
# fiz uma pequena alteração no código para ler o csv direto do google drive
from google.colab import drive #código adicionado

drive.mount('/content/drive') #código adicionado
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/notebooks-atividades-praticas/diabetes.csv' )#código alterado

data.head() # display dataset first lines

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# Separate the class from other attributes
X = data.drop("Outcome", axis=1)
Y = data["Outcome"]

Documentação de *train_test_split*: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html <br>
A função divide os dados em partições de treino e teste, de acordo com a proporção especificada pelo parâmetro *test_size*. <br>
O parâmetro *random_state* é usado para deixar os resultados reproduzíveis para fins de avaliação do exercício.

In [ ]:
# Preparing the dataset for training and test
from sklearn.model_selection import train_test_split
train_set_X, test_set_X, train_set_Y, test_set_Y = train_test_split(X, Y, test_size=0.20, random_state=7)

Padronize os atributos usando a média e a variância dos dados

Dica: use a função *fit_transform*: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

### INICIE O CÓDIGO AQUI ### (2 linhas de código)
train_set_X = scaler.fit_transform(train_set_X)
test_set_X  = scaler.transform(test_set_X)
### TERMINE O CÓDIGO AQUI ###

In [ ]:
# Checking
print("train_set_X:\n", train_set_X[:2,:])
print("\ntest_set_X:\n", test_set_X[:2,:])

train_set_X:
 [[ 0.35483802 -0.370418    0.16624635  1.40032403 -0.01279543  0.49863797
  -0.61786951  0.01064085]
 [-0.54794048 -0.55620696  0.9024924   0.96548604  0.40069886  1.70739891
  -1.0345765  -0.86049025]]

test_set_X:
 [[-0.84886664 -0.92778487 -0.35964369 -0.52538709 -0.30683581 -0.64582717
   0.35342545 -0.77337714]
 [ 0.95669035  1.89001433  0.7973144   0.03369033  1.06228972  0.47291965
   0.37180958  1.57867683]]


**SAÍDA ESPERADA:** <br>
train_set_X: <br>
 [[ 0.35483802 -0.370418    0.16624635  1.40032403 -0.01279543  0.49863797  -0.61786951  0.01064085] <br>
 [-0.54794048 -0.55620696  0.9024924   0.96548604  0.40069886  1.70739891  -1.0345765  -0.86049025]] <br>
<br>
test_set_X: <br>
[[-0.83168001 -1.14367855 -0.39723824 -0.57849223 -0.36848171 -0.47210798  0.22800925 -0.84284936] <br>
 [ 0.8647269   1.8633592   0.67258938  0.00976179  0.72124393  0.58740671  0.24525553  1.28629259]] <br>

 ---

Obtenha o número de atributos e o número de exemplos de treinamento

Dica: use a função *shape*: https://numpy.org/devdocs/reference/generated/numpy.shape.html

In [ ]:
### INICIE O CÓDIGO AQUI ### (2 linhas de código)
n = train_set_X.shape[1]        # number of attributes
m = train_set_X.shape[0]         # number of training examples
### TERMINE O CÓDIGO AQUI ###

print ("Number of attributes: n = " + str(n))
print ("Number of training examples: m = " + str(m))
print ("Train set X shape: " + str(train_set_X.shape))
print ("Train set Y shape: " + str(train_set_Y.shape))
print ("Test set X shape: " + str(test_set_X.shape))
print ("Test set Y shape: " + str(test_set_Y.shape))

Number of attributes: n = 8
Number of training examples: m = 614
Train set X shape: (614, 8)
Train set Y shape: (614,)
Test set X shape: (154, 8)
Test set Y shape: (154,)


**SAÍDA ESPERADA**: <br>
Number of attributes: n = 8 <br>
Number of training examples: m = 614 <br>
Train set X shape: (614, 8) <br>
Train set Y shape: (614,) <br>
Test set X shape: (154, 8) <br>
Test set Y shape: (154,) <br>

---

## Definição do Modelo

Crie um modelo em Keras com a seguinte configuração:
- Camada de entrada: no formato dos dados de entrada do problema
- Camada 1: 3 neurônios, função de ativação *Tanh*
- Camada 2: 5 neurônios, função de ativação *Tanh*
- Camada 3: 3 neurônios, função de ativação *Tanh*
- Camada 4 (saída): 1 neurônio, função de ativação *Sigmoid*

Dica 1: use a classe *Model*: https://keras.io/api/models/model/ <br>
Dica 2: veja as funções de ativação disponíveis: https://keras.io/api/layers/activations/

In [ ]:
### INICIE O CÓDIGO AQUI ### (6 linhas de código)
inputs = keras.Input(shape=(n,)) #camada de entrada
x = keras.layers.Dense(3, activation='tanh')(inputs) #camada 1
x = keras.layers.Dense(5, activation='tanh')(x)      #camada 2
x = keras.layers.Dense(3, activation='tanh')(x)      #camada 3
outputs = keras.layers.Dense(1, activation='sigmoid')(x) #camada 4 (camada de saída)
model = keras.Model(inputs=inputs, outputs=outputs)
### TERMINE O CÓDIGO AQUI ###

In [ ]:
# Checking
processed_data = model(train_set_X)
print(processed_data.shape)

(614, 1)


**SAÍDA ESPERADA**: <br>
(614, 1)

---

In [ ]:
# Prints a summary of the network, showing its architecture and parameters.
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │            27 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69 (276.00 B)

 Trainable params: 69 (276.00 B)

 Non-trainable params: 0 (0.00 B)

**SAÍDA ESPERADA**: <br>
Confira a configuração da rede e o total de parâmetros = 69

---

## Compilação do Modelo

Compile o model usando os seguintes parâmetros:
- Função de perda: mean_absolute_error
- Otimizador: RMSprop
- Métricas: accuracy, Precision, Recall

Dica 1: use a função *compile*: https://keras.io/api/models/model_training_apis/ <br>
Dica 2: relação de funções de perda: https://www.tensorflow.org/api_docs/python/tf/keras/losses <br>
Dica 3: relação de otimizadores: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers <br>
Dica 4: relação de métricas: https://keras.io/api/metrics/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)
model.compile(optimizer="rmsprop", loss="mean_absolute_error", metrics=["accuracy","Precision","recall"])
### TERMINE O CÓDIGO AQUI ###

## Treinamento do Modelo

Ajusta o modelo aos dados de treinamento.
Devem ser fornecidos os dados de treinamento, o número de épocas (iterações) e o tamanho do lote (batch). Uma época é composta por uma única passagem por todos os exemplos do conjunto de treino. O tamanho do lote define o número de amostras (exemplos) a serem consideradas pelo modelo antes de atualizar os pesos. Assim, uma época é composta por um ou mais lotes.

Efetue o treinamento do modelo usando os seguintes parâmetros:
- Tamanho do lote: 64
- Número de épocas: 1000

Dica: use a função *fit*: https://keras.io/api/models/model_training_apis/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)
history = model.fit(train_set_X, train_set_Y, epochs=1000, batch_size=64)
### TERMINE O CÓDIGO AQUI ###
print(history.history)  # print per-epoch timeseries of metrics values

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - Precision: 0.2812 - accuracy: 0.4251 - loss: 0.5136 - recall: 0.3771
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.3313 - accuracy: 0.5315 - loss: 0.4962 - recall: 0.3859 
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.3312 - accuracy: 0.5739 - loss: 0.4840 - recall: 0.3840 
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.3774 - accuracy: 0.5888 - loss: 0.4831 - recall: 0.3012 
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.4456 - accuracy: 0.6170 - loss: 0.4771 - recall: 0.3497 
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.5133 - accuracy: 0.6557 - loss: 0.4614 - recall: 0.3669 
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.5037 - accuracy: 0.6423 - loss: 0.4599 - recall: 0.3097 
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - Precision: 0.4928 - accuracy: 0.6761 - loss: 0.4430 - recall: 0.3025 
Epoch 9/

**SAÍDA ESPERADA**: <br>
Na época 1000, tem-se o seguinte resultado (aproximado): <br>
loss: 0.1938 - accuracy: 0.8062 - precision: 0.7644 - recall: 0.6303

---

## Avaliação do Modelo

Avalie o desempenho da rede no conjunto de teste.

Dica: use a função *evaluate*: https://keras.io/api/models/model_training_apis/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)
loss, acc, prec, rec = model.evaluate(test_set_X, test_set_Y)
### TERMINE O CÓDIGO AQUI ###
print("Loss: %.2f" % loss, "\nAccuracy: %.2f" % acc, "\nPrecision: %.2f" % prec, "\nRecall: %.2f" % rec)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - Precision: 0.7507 - accuracy: 0.7923 - loss: 0.2080 - recall: 0.6383
Loss: 0.23 
Accuracy: 0.77 
Precision: 0.73 
Recall: 0.61


**SAÍDA ESPERADA**: <br>
Valores aproximados:

Loss: 0.21  <br>
Accuracy: 0.79  <br>
Precision: 0.76  <br>
Recall: 0.61

---

## Predição

Apresente a predição do conjunto de teste.

Dica: use a função *predict*: https://keras.io/api/models/model_training_apis/

In [ ]:
### INICIE O CÓDIGO AQUI ### (1 linha de código)
predictions = model.predict(test_set_X)
### TERMINE O CÓDIGO AQUI ###
print("Predictions: ", [round(x[0]) for x in predictions])
print("\nCorrect:     ", [round(x) for x in test_set_Y])

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predictions:  [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]

Correct:      [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]


**SAÍDA ESPERADA**: <br>
Valores aproximados: <br>
Predictions:  [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0]

Correct:      [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]

---

## Desafio

Modifique a configuração da sua rede e/ou os parâmetros dos métodos com o objetivo de melhorar os resultados das métricas no conjunto de teste. Experimente várias opções. <br>
Adicione abaixo a nova sequência de código que alcançou o melhor resultado.

A ideia das alterações foi mudar as funções de ativação das camadas ocultas para "ReLU", mudar o otimizador para "adam", e a função de perda para "binary cross-entropy". Além disso, o batch size foi alterado para 32, o número de epochs foi reduzido para 250 (pois com mais epochs obtive problemas com overfiting), além de ter diminuido o número de camada e aumentado o número de neurônios. Foi possível observar um ligeiro aumento tanto na acurácia quanto na precisão.

In [ ]:
### INICIE O CÓDIGO AQUI ### (várias linhas de código / várias células)
inputs2 = keras.Input(shape=(n,)) #camada de entrada
x2 = keras.layers.Dense(8, activation='relu')(inputs2) #camada 1
#x2 = keras.layers.Dense(8, activation='relu')(x2)      #camada 2
x2 = keras.layers.Dense(16, activation='relu')(x2)      #camada 3
outputs2 = keras.layers.Dense(1, activation='sigmoid')(x2) #camada 4 (camada de saída)
model2 = keras.Model(inputs=inputs2, outputs=outputs2)

model2.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 233 (932.00 B)

 Trainable params: 233 (932.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model2.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy","Precision","recall"])

In [ ]:
history2 = model2.fit(train_set_X, train_set_Y, epochs=250, batch_size=32)

print(history2.history)

Epoch 1/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - Precision: 0.2112 - accuracy: 0.4014 - loss: 0.7268 - recall: 0.2646
Epoch 2/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Precision: 0.3081 - accuracy: 0.5362 - loss: 0.6949 - recall: 0.1826
Epoch 3/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - Precision: 0.5571 - accuracy: 0.6715 - loss: 0.6717 - recall: 0.0965
Epoch 4/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Precision: 0.7813 - accuracy: 0.6604 - loss: 0.6590 - recall: 0.0625         
Epoch 5/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Precision: 0.7222 - accuracy: 0.6872 - loss: 0.6314 - recall: 0.0696 
Epoch 6/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Precision: 0.7995 - accuracy: 0.6418 - loss: 0.6319 - recall: 0.0448         
Epoch 7/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Precision: 0.7769 - accuracy: 0.6735 - loss: 0.6054 - recall: 0.0703         
Epoch 8/250
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Precision: 0.8205 - accuracy: 0.6834 - loss: 0.5970 - recall: 0.

In [ ]:
loss, acc, prec, rec = model2.evaluate(test_set_X, test_set_Y)

print("Loss: %.2f" % loss, "\nAccuracy: %.2f" % acc, "\nPrecision: %.2f" % prec, "\nRecall: %.2f" % rec)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - Precision: 0.7944 - accuracy: 0.8274 - loss: 0.4494 - recall: 0.7158 
Loss: 0.50 
Accuracy: 0.82 
Precision: 0.81 
Recall: 0.67


In [ ]:
predictions2 = model2.predict(test_set_X)
### TERMINE O CÓDIGO AQUI ###
print("Predictions: ", [round(x[0]) for x in predictions2])
print("\nCorrect:     ", [round(x) for x in test_set_Y])
### TERMINE O CÓDIGO AQUI ###

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Predictions:  [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

Correct:      [0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]


# Fim

Parabéns! Você efetuou todos os passos para criar uma rede neural com várias camadas para um problema de classificação binária.

---